# Geocode Chicago Red Light Camera locations
* geocode locations from the Propublica data set (https://www.propublica.org/datastore/dataset/chicago-parking-ticket-data)

### Set up dependencies

In [1]:
# Import dependencies and key
import requests
import json
import pandas as pd
import gmaps
from IPython.display import Image

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# address file to read (generated from 'ChicagoTickets.ipynb')
csv_file = "Chicago_Count_by_Location.csv"

### Read Addresses

In [2]:
# read the file created in ChicagoTickets.ipynb
df = pd.read_csv(csv_file)
df.head()

,address,Total,First Ticket Date,Last Ticket Date,Days,Daily Avg
0,1 E 63RD ST,12038,3/3/2014,4/27/2018,1517,7.935399
1,1 E 75TH STREET,15159,3/1/2014,4/27/2018,1519,9.979592
2,1 E 79TH STREET,38135,3/1/2014,4/27/2018,1519,25.105332
3,1 N ASHLAND AVE,7229,3/1/2014,4/27/2018,1519,4.759052
4,1 N HALSTED STREET,5696,3/1/2014,4/27/2018,1519,3.749835


### Geocode and return lat/longitude

In [3]:
# base_url
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# params
params = {"key": gkey}

for index, row in df.iterrows():
    # display message
    print(f"Geocoding {row['address']}")
    
    # add address to params
    params['address'] = f"{row['address']}, Chicago IL"
    
    # make request
    r = requests.get(base_url, params=params).json()
    
    # convert data to data frame
    try:
        # grab lat/long from 'geometry'
        df.loc[index, "Lat"] = r["results"][0]["geometry"]["location"]["lat"]
        df.loc[index, "Lng"] = r["results"][0]["geometry"]["location"]["lng"]
        
        # grab zip code from 'address_comonents'
        for i in r["results"][0]['address_components']:
            if i['types'][0] == "postal_code":
                df.loc[index, 'zipcode'] = i['short_name']
                break
    except:
        print(f"Error geocoding {row['address']}")

Geocoding 1 E 63RD ST
Geocoding 1 E 75TH STREET
Geocoding 1 E 79TH STREET
Geocoding 1 N ASHLAND AVE
Geocoding 1 N HALSTED STREET
Geocoding 1 N HAMLIN AVENUE
Geocoding 1 S CENTRAL AVENUE
Geocoding 1 S HALSTED STREET
Geocoding 1 S WESTERN AVE
Geocoding 100 E JACKSON BLVD
Geocoding 100 E ONTARIO ST
Geocoding 100 N CICERO AVENUE
Geocoding 100 W CHICAGO AVENUE
Geocoding 1000 W FOSTER AVE
Geocoding 1000 W HOLLYWOOD AVE
Geocoding 10300 S HALSTED STREE
Geocoding 10318 S INDIANAPOLIS
Geocoding 1110 S PULASKI RD
Geocoding 11100 S HALSTED ST
Geocoding 11100 S VINCENNES AVE
Geocoding 1111 N HUMBOLDT
Geocoding 11144 S VINCENNES
Geocoding 11153 S VINCENNES
Geocoding 1117 S PULASKI RD
Geocoding 1142 W IRVING PARK
Geocoding 115 N OGDEN
Geocoding 11500 S HALSTED STREE
Geocoding 11900 S HALSTED
Geocoding 1200 N ASHLAND AVENUE
Geocoding 1200 N HALSTED
Geocoding 1200 N PULASKI ROAD
Geocoding 1200 S CANAL STREET
Geocoding 1200 S KOSTNER
Geocoding 1200 S PULASKI ROAD
Geocoding 1200 W DEVON AVE
Geocoding 120

In [4]:
# Save results to csv
df.to_csv("Chicago_Count_by_Location_LatLng.csv", index=False)

### Tickets Heat Map

In [5]:
# Remove addresses that could not be geocoded
df_loc = df.dropna()
df_loc.count()

address              465
Total                465
First Ticket Date    465
Last Ticket Date     465
Days                 465
Daily Avg            465
Lat                  465
Lng                  465
zipcode              465
dtype: int64

In [7]:
# Data to plot
locations = df_loc[["Lat", "Lng"]]
weights = df_loc['Daily Avg']

# Plot Heatmap
fig = gmaps.figure(
    layout={
        'width': '800px',
        'height': '500px'
    }
)

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=weights, 
    dissipating=False,
    max_intensity=max(weights),
    point_radius=0.02)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

In [16]:
# add top 5 cameras with most tickets
df_top5 = df_loc.nlargest(5,'Daily Avg')[['address', 'Daily Avg', 'Lat', 'Lng']]
df_top5['Daily Avg'] = df_top5['Daily Avg'].round(0)
df_top5

,address,Daily Avg,Lat,Lng
301,4909 N CICERO AVE,200.0,41.970532,-87.747724
275,445 W 127TH,136.0,41.663262,-87.633202
153,OGDEN and SACRAMENTO,118.0,41.859124,-87.701571
260,4124 W FOSTER AVE,111.0,41.975533,-87.731463
137,2705 W IRVING PARK,93.0,41.953911,-87.695747


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Loc</dt><dd>{address}</dd>
<dt>Daily Tix #</dt><dd>{Daily Avg}</dd>
</dl>
"""

# Store the DataFrame Row
camera_info = [info_box_template.format(**row) for index, row in df_top5.iterrows()]
locations = df_top5[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(
    locations,
    info_box_content=camera_info
)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

![title](DailyAvgHeatMap.png)